In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import pandas as pd
import skimage.draw
import matplotlib.pyplot as plt

In [ ]:
import keras

In [ ]:
keras.__version__

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../Mask_RCNN")
 
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn import model as modellib, utils
 
# Path to trained weights file
# COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
 
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "damage"
    
    GPU_COUNT = 1
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1
 
    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + toy
 
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 30
 
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

# Dataset Class

In [ ]:

annotations1 = json.load(open(os.path.join("./dataset_test1/", "via_region_data.json")))

annotations = list(annotations1.values())  

In [ ]:
annotations = [a for a in annotations if a['regions']]

In [ ]:
a = annotations[0]
polygons = [r["shape_attributes"] for r in a["regions"]]
damages = [r["region_attributes"]["Severity"] for r in a['regions']]

In [ ]:
polygons

In [ ]:
damage_to_id = {"Light Scratches": 1,
                "Moderate Scratches": 2,
                "Light Dent": 3,
                "Moderate Dent": 4,
                "Severe Dent": 5 
               }

In [ ]:
num_ids = [damage_to_id[list(damage.keys())[0]] for damage in damages]

In [ ]:
num_ids

In [ ]:
class CustomDataset(utils.Dataset):
 
    def load_custom(self, input_dir, subset):
        """Load a subset of the bottle dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("damage", 1, "scratches")
        self.add_class("damage", 2, "dents")
        #self.add_class("damage", 3, "light_dent")
        #self.add_class("damage", 4, "moderate_dent")
        #self.add_class("damage", 5, "severe_dent")
        
        assert subset in ["train", "val"]
        
        dataset_dir = os.path.join(input_dir, subset)
 
        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
#         annotations1 = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations1 = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys
 
        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        count = 1
        # Add images
        for a in annotations:
            print(count)
            count+=1
            
            print(a["filename"])
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r["shape_attributes"] for r in a["regions"]]
            damages = [r["region_attributes"]["Severity"] for r in a['regions']]
            print("Damages:",damages)
            damage_to_id = {"Light Scratches": 1,
                            "Moderate Scratches": 1,
                            "Light Dent": 2,
                            "Moderate Dent": 2,
                            "Severe Dent": 2 
                           }
            # key = tuple(name_dict)
            num_ids = [damage_to_id[list(damage.keys())[0]] for damage in damages]
 
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
 
            self.add_image(
                "damage",  
                image_id=a['filename'],  # file name has a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)
    
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a bottle dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "damage":
            return super(self.__class__, self).load_mask(image_id)
 
        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "damage":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
 
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

In [ ]:
train_set = CustomDataset()

In [ ]:
train_set.load_custom("./dataset_train1/", "train")

## Testing the Train Dataset

In [ ]:
image_id = 4

image = train_set.load_image(image_id)
print(image.shape)

mask, class_id = train_set.load_mask(image_id)
print(mask.shape)

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
plt.imshow(mask)
plt.show()

# Training Function & Other Functions

In [ ]:
dataset_dir = "./dataset_train2/"

In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(dataset_dir)
    dataset_train.prepare()
 
    # Validation dataset
#     dataset_val = CustomDataset()
#     dataset_val.load_custom(dataset_dir)
#     dataset_val.prepare()
 
    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=2,
                layers='heads')

In [ ]:
dataset_train = CustomDataset()
dataset_train.load_custom(dataset_dir, "train")
dataset_train.prepare()

In [ ]:
dataset_test = CustomDataset()
dataset_test.load_custom(dataset_dir, "val")
dataset_test.prepare()

In [ ]:
config = CustomConfig()

In [ ]:
logs = "./log_v7/"

In [ ]:
model = modellib.MaskRCNN(mode="training", config = config, model_dir = logs)

In [ ]:
# Loading the COCO weights
COCO_WEIGHTS_PATH = "mask_rcnn_damage_0040.h5"

model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"
])

In [ ]:
history = model.train(dataset_train, dataset_test,
                learning_rate=config.LEARNING_RATE,
                epochs=50,
                layers='all')